In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
rx = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

In [ ]:
 from tqdm.auto import tqdm

In [ ]:
rxv = []
for text in tqdm(rx.text):
    rxv.append(nlp(text).vector)

In [ ]:
rxv = np.stack(rxv)

# Clearly data has loads of features, which might be hard to visualise, we should employ dimensionality reduction methods - perhaps UMAP!
import umap

import warnings
from numba.errors import NumbaPerformanceWarning
warnings.simplefilter('ignore', category=NumbaPerformanceWarning) # suppressing those annoyeing waranings

imap = umap.UMAP()
imap.fit(rxv[::10])


In [ ]:
rxd = imap.transform(rxv)
rxd = pd.DataFrame(rxd, columns=['x','y'], index=rx.index)

In [ ]:
rx = rx.join(rxd)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(20,10))

plt.scatter(rx.x, rx.y, c =rx.target)

In [ ]:
ex = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

exv = []
for text in tqdm(ex.text):
    exv.append(nlp(text).vector)
exv = np.stack(exv)

exd = imap.transform(exv)
exd = pd.DataFrame(exd, columns=['x','y'], index=ex.index)

ex = ex.join(exd)

plt.figure(figsize=(20,10))
plt.scatter(ex.x, ex.y, s=10)

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.stats import mode

neighbors = NearestNeighbors()

neighbors.fit(rx[['x','y']], rx.target)


_,ind = neighbors.kneighbors(ex[['x','y']], n_neighbors=3)

etarget = [[rx.iloc[i].target for i in ii]  for ii in ind.squeeze()] 


ey = [mode(l).mode[0] for l in etarget]

ex['target'] = ey

plt.figure(figsize=(20,10))
plt.scatter(ex.x, ex.y, c=ex.target)

plt.scatter(rx.x, rx.y, c =rx.target, alpha=0.01,s=150 )

In [ ]:
submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

submission.target = ex.target

submission.to_csv('submission.csv', index=False)